# Evaluation rerun

the evaluation failed for some for questions, but we have the model answers so I want to rerun the evaluation

In [ ]:
from cat_tools.eval import similarity_scorer
import weave
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
client = weave.init("smart-drying-unitus/declarative-eval")

weave: weave version 0.52.6 is available!  To upgrade, please run:
weave:  $ pip install weave --upgrade
weave: Logged in as Weights & Biases user: smassaro-unitus.
weave: View Weave data at https://wandb.ai/smart-drying-unitus/declarative-eval/weave


In [ ]:
calls = list(client.get_calls(
    filter={"trace_roots_only": True},
    query={"$expr":{"$and":[{"$gt":[{"$getField":"started_at"},{"$literal":1756670280.366}]},{"$or":[{"$eq":[{"$getField":"id"},{"$literal":"01990fbd-ae07-74cc-9b8e-9e5ae323d958"}]},{"$eq":[{"$getField":"id"},{"$literal":"01991061-7220-7a2a-aa03-a14bd6c8a003"}]},{"$eq":[{"$getField":"id"},{"$literal":"019910d8-995a-78ed-8151-afbd5a5e4223"}]},{"$eq":[{"$getField":"id"},{"$literal":"0199115d-6ac3-7422-815a-c2b56b7decae"}]},{"$eq":[{"$getField":"id"},{"$literal":"01991448-bdfd-762d-ab6c-c4851521d521"}]},{"$eq":[{"$getField":"id"},{"$literal":"019916f6-9b85-7a78-90d5-f4390da1c86a"}]},{"$eq":[{"$getField":"id"},{"$literal":"0199175a-03ae-7809-8292-f22c2320f1ee"}]},{"$eq":[{"$getField":"id"},{"$literal":"019917ed-01ca-7bad-ab97-fe987fbc6d1d"}]},{"$eq":[{"$getField":"id"},{"$literal":"019918d8-2ad3-7931-8fd8-6077a277388b"}]},{"$eq":[{"$getField":"id"},{"$literal":"01991c77-6db6-734d-9b02-19cea3454c16"}]}]}]}},
    sort_by=[{"field":"started_at","direction":"desc"}],
))

In [ ]:
def parse_calls(calls):
    """Return a DataFrame with:
    - input
    - expected output
    - cleaned output
    - raw weave call object
    - similarity score (if available)
    - model_name
    - has_declative_memory
    """
    outs = []
    for call in tqdm(list(calls)):
        model_name = call.inputs['model'].name
        has_mem =  'NO memory' not in call.display_name
        for row in call.children():
            out = {}
            if inp := row.inputs.unwrap():
                if inp := inp.get('example', False):
                        out['question_id']= inp['id']
                        out['question']= inp['prompt']
                        out['expected_answer']= inp['target']
            if output := row.output:
                output = output.unwrap()
                if 'output' in output:
                    if 'text_clean' in output['output']:
                        out['model_answer'] = output['output']['text_clean']
                    if 'scores' in output:
                        if 'CatEmbeddingSimilarityScorer' in output['scores']:
                            out['similarity_score'] = output['scores']['CatEmbeddingSimilarityScorer']['similarity_score']

            out['model_name'] = model_name
            out['weave_call'] = row
            out['has_declarative_memory'] = has_mem
            outs.append(out)
    return pd.DataFrame(outs)

In [ ]:
df = parse_calls(calls)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df

,question_id,question,expected_answer,model_name,weave_call,model_answer,has_declarative_memory,similarity_score
0,0.0,Che cos'è la disidratazione a flusso di aria c...,La disidratazione a flusso di aria calda è un ...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,NaN,NaN,NaN
1,1.0,Qual è l'obiettivo principale del processo di ...,L'obiettivo principale del processo di disidra...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,NaN,NaN,NaN
2,2.0,Quali sono i principali parametri da controlla...,Durante la disidratazione a flusso di aria cal...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,NaN,NaN,NaN
3,3.0,Quali fenomeni fisici avvengono durante la dis...,Durante la disidratazione di un alimento avven...,deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,NaN,NaN,NaN
4,4.0,Quali vantaggi offre la disidratazione oltre a...,"Oltre alla conservazione, la disidratazione of...",deepseek_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3201,332.0,Come vengono caricati e scaricati i contenitor...,I contenitori del prodotto vengono caricati e ...,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,I contenitori vengono caricati e scaricati att...,True,0.983811
3202,333.0,In che modo questo essiccatore gestisce il ric...,Questo essiccatore gestisce il ricircolo dell'...,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,Gestisce il ricircolo tramite un passaggio ret...,True,0.924412
3203,334.0,Perché le pareti dell'essiccatore sono realizz...,Le pareti dell'essiccatore sono realizzate con...,openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,"Per garantire isolamento, efficienza e durata....",True,0.892559
3204,335.0,È possibile impostare diverse temperature per ...,"Sì, il pannello di controllo di questo essicca...",openai_best,WeaveObject(Call(_op_name='weave:///smart-dryi...,Sì. Il pannello di controllo consente di impos...,True,0.956343


In [ ]:
call = calls[0]

In [ ]:
call.display_name

'09-03 15:20 Eval - deepseek_best NO memory'

In [ ]:
import polars as pl
from polars import col as c

In [ ]:
df.group_by['model_name']

In [ ]:
df_pl = pl.from_pandas(df.drop(columns=['weave_call']))

In [ ]:
df_pl.group_by(c.model_name, c.has_declarative_memory).agg(frac_answers = c.model_answer.is_not_null().sum() / 336, frac_scores = c.similarity_score.is_not_null().sum() / 336).sort(c.has_declarative_memory, c.model_name)

model_name,has_declarative_memory,frac_answers,frac_scores
str,bool,f64,f64
"""deepseek_best""",false,0.0,0.0
"""gemini_best""",false,0.910714,0.910714
"""gemma_best""",false,0.0,0.0
"""openai_best""",false,0.404762,0.404762
"""qwen_best""",false,0.0,0.0
"""deepseek_best""",true,0.892857,0.892857
"""gemini_best""",true,0.991071,0.991071
"""gemma_best""",true,0.997024,0.997024
"""openai_best""",true,0.985119,0.985119


In [ ]:
row = list(calls[-1].children())[0]

In [ ]:
row

WeaveObject(Call(_op_name='weave:///smart-drying-unitus/declarative-eval/op/Evaluation.predict_and_score:NbvZTtBp9EjaNqvaUHIhY0JwtlXanwJRjGThby0Fa1k', trace_id='01990fbd-ae07-796c-ad00-c3ef519caf06', project_id='smart-drying-unitus/declarative-eval', parent_id='01990fbd-ae07-74cc-9b8e-9e5ae323d958', inputs={'self': ObjectRef(entity='smart-drying-unitus', project='declarative-eval', name='09-03-15-20-Eval', _digest='kSGrXwNdSG56q5FHh9vdcedEWhdxNpfeqxRQCKCHWDo', _extra=()), 'model': ObjectRef(entity='smart-drying-unitus', project='declarative-eval', name='openai_best', _digest='Ov47WVbybuLCSPEdzQ2mGeEEY9BeNU8K3AoFaOdRZ5s', _extra=()), 'example': ObjectRef(entity='smart-drying-unitus', project='declarative-eval', name='Dataset', _digest='QeX53264vZm9JLDS7fxpPpj3SHYYLHYHHarVn7N3R3s', _extra=('attr', 'rows', 'id', '3dZooKbMOO60556DXrgtG3ksaHac5YZXepSd9sZqfl4'))}, id='01990fbd-ae09-79cf-8b1d-3f7de8e15f62', output={'output': {'user_id': 'user', 'when': 1756905754.450417, 'who': 'AI', 'text': 

In [ ]:
row.inputs.unwrap()['example']

{'id': 0,
 'prompt': "Che cos'è la disidratazione a flusso di aria calda applicata agli alimenti?",
 'input': "Che cos'è la disidratazione a flusso di aria calda applicata agli alimenti?",
 'context': "La disidratazione a flusso di aria calda è un processo tecnologico che consiste nell'applicare calore a un alimento in condizioni controllate. Lo scopo è rimuovere gran parte dell'acqua contenuta al suo interno tramite evaporazione, trasformandola in vapore acqueo che viene poi allontanato.",
 'target': "La disidratazione a flusso di aria calda è un processo tecnologico che consiste nell'applicare calore a un alimento in condizioni controllate. Lo scopo è rimuovere gran parte dell'acqua contenuta al suo interno tramite evaporazione, trasformandola in vapore acqueo che viene poi allontanato.",
 'question_id': 0}

In [ ]:
row.output.unwrap()['scores']

{'HallucinationFreeScorer': {'chain_of_thought': "I need to check if the information in the <output> is supported by the <input_data>. The <input_data> provides a very brief, high-level definition of hot air dehydration. It states it's a process of applying controlled heat to food to remove water through evaporation. The <output> gives a much more detailed and comprehensive explanation, covering topics like convective drying, water activity (aw), specific process phases, key parameters with numerical values (e.g., temperatures for different foods), effects on quality, typical equipment, common applications, and pros and cons. I will compare the details in the output with the simple definition in the input. Almost all the information in the output, such as specific temperatures, equipment types, and effects on food quality, is not present in the input. Therefore, the output contains a significant amount of information not supported by the input data.",
  'reasonings': [{'hallucination_t